In [ ]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import json
import requests
import datetime as dt
import schedule
import time
import tweepy
def run_algo():
    
    url = 'https://api.binance.com/api/v3/klines'
    symbol = 'ETHUSDT'
    interval = '1d'
    start = str(int(dt.datetime(2022,1,1).timestamp()*1000))  #data start from: 2017,8,16
    end = str(int(dt.datetime(2022,12,30).timestamp()*1000))
    par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 5000}
    eth = pd.DataFrame(json.loads(requests.get(url, params= par).text))
    eth.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    eth.drop(columns=['open', 'high', 'low', 'volume', 'close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'], inplace = True) #we will only use close price
    eth.index = [dt.datetime.fromtimestamp(x/1000.0) for x in eth.datetime]
    eth=eth.astype(float)

    #algo
    ma = 10
    eth['returns'] = np.log(eth["close"]).diff()
    eth['ma'] = eth['close'].rolling(ma).mean()
    eth['ratio'] = eth['close'] / eth['ma']
    
    percentiles = [2.5, 7.5, 45, 87.5, 92.5]
    p = np.percentile(eth['ratio'].dropna(), percentiles)
    short = p[-1]
    long = p[0]
    eth['position'] = np.where(eth.ratio > short, -1, np.nan)
    eth['position'] = np.where(eth.ratio < long, 1, eth['position'])
    eth['position'] = eth['position'].ffill()
    eth['entry'] =  eth.position.diff()
    
    #plotting
    eth['10d-ma'] = eth['close'].rolling(10).mean().shift()
    plt.rcParams['figure.figsize'] = 19, 9
    plt.grid(True, alpha = .3)
    days = 360; days = -(days)
    plt.plot(eth.iloc[days:]['close'], label = 'btc')
    plt.plot(eth.iloc[days:]['10d-ma'], label = '10d-ma')
    plt.plot(eth[days:].loc[eth.entry == 2].index, eth[days:]['close'][eth.entry == 2], '^', color = 'g', markersize = 9)
    plt.plot(eth[days:].loc[eth.entry == -2].index, eth[days:]['close'][eth.entry == -2], 'v', color = 'r', markersize = 9)
    plt.legend(loc=3);
    plt.savefig('eth_1h_meanrev.jpg')
    plt.clf()

    ############################################## V1

    if(eth['position'][-1] != eth['position'][-2]):
        if(eth['position'][-1] == 1):
            pos='eth/usdt long  @ '
            price=str(eth['close'][-1])
            algo = '\n(mean reverting 1d)'
            text=pos+price+algo

        else:
            pos='eth/usdt short  @ '
            price=str(eth['close'][-1])
            entry=pos+price
            algo = '\n(mean reverting 1d)'
            text=pos+price+algo
            
        #posting on twitter
        # Enter your API credentials
        consumer_key = 'key'
        consumer_secret = 'secret'
        access_token = 'token'
        access_token_secret = 'token_secret'

        # Authenticate with the Twitter API using your API credentials
        auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)

        # Connect to the Twitter API using the authenticate handler
        api = tweepy.API(auth)

        # Open the image file
        image_path = 'btc_1h_meanrev.jpg'
        image = open(image_path, 'rb')

        # Set the status update text
        status_text = text

        # Upload the image to Twitter
        upload_response = api.media_upload(image_path)
        media_id = upload_response.media_id_string

        # Publish the tweet with the image and status update
        status = api.update_status(status=status_text, media_ids=[media_id])
    
schedule.every().day.at("21:00").do(run_algo)
while True:
    schedule.run_pending()
    time.sleep(0.999)